# Forest Covertype

Classification of pixels into 7 forest cover types based on attributes such as elevation, aspect, slope, hillshade, soil-type, and more.
Dataset available [here](https://archive.ics.uci.edu/dataset/31/covertype).

## General Information
Predicting forest cover type from cartographic variables only (no remotely sensed data).  The actual forest cover type for a given observation (30 x 30 meter cell) was determined from US Forest Service (USFS) Region 2 Resource Information System (RIS) data.  Independent variables were derived from data originally obtained from US Geological Survey (USGS) and USFS data.  Data is in raw form (not scaled) and contains binary (0 or 1) columns of data for qualitative independent variables (wilderness areas and soil types).

This study area includes four wilderness areas located in the Roosevelt National Forest of northern Colorado.  These areas represent forests with minimal human-caused disturbances, so that existing forest cover types are more a result of ecological processes rather than forest management practices.

Some background information for these four wilderness areas: Neota (area 2) probably has the highest mean elevational value of the 4 wilderness areas. Rawah (area 1) and Comanche Peak (area 3) would have a lower mean elevational value, while Cache la Poudre (area 4) would have the lowest mean elevational value. 

As for primary major tree species in these areas, Neota would have spruce/fir (type 1), while Rawah and Comanche Peak would probably have lodgepole pine (type 2) as their primary species, followed by spruce/fir and aspen (type 5). Cache la Poudre would tend to have Ponderosa pine (type 3), Douglas-fir (type 6), and cottonwood/willow (type 4).  

The Rawah and Comanche Peak areas would tend to be more typical of the overall dataset than either the Neota or Cache la Poudre, due to their assortment of tree species and range of predictive variable values (elevation, etc.)  Cache la Poudre would probably  be more unique than the others, due to its relatively low  elevation range and species composition. 

## Additional Variable Information
Given is the attribute name, attribute type, the measurement unit and a brief description.  The forest cover type is the classification  problem.  The order of this listing corresponds to the order of numerals along the rows of the database.

|Name | Data Type | Measurement | Description|
|-----|-----------|-------------|------------|
|Elevation | quantitative |meters | Elevation in meters|
|Aspect | quantitative | azimuth | Aspect in degrees azimuth|
|Slope | quantitative | degrees | Slope in degrees|
|Horizontal_Distance_To_Hydrology | quantitative | meters | Horz Dist to nearest surface water features|
|Vertical_Distance_To_Hydrology | quantitative | meters | Vert Dist to nearest surface water features|
|Horizontal_Distance_To_Roadways | quantitative | meters | Horz Dist to nearest roadway|
|Hillshade_9am | quantitative | 0 to 255 index | Hillshade index at 9am, summer solstice|
|Hillshade_Noon | quantitative | 0 to 255 index | Hillshade index at noon, summer soltice|
|Hillshade_3pm | quantitative | 0 to 255 index | Hillshade index at 3pm, summer solstice|
|Horizontal_Distance_To_Fire_Points | quantitative | meters | Horz Dist to nearest wildfire ignition points|
|Wilderness_Area (4 binary columns) | qualitative | 0 (absence) or 1 (presence) | Wilderness area designation|
|Soil_Type (40 binary columns) | qualitative | 0 (absence) or 1 (presence) | Soil Type designation|
|Cover_Type (7 types) | integer | 1 to 7 | Forest Cover Type designation|

## Class labels
Spruce/Fir, Lodgepole Pine, Ponderosa Pine, Cottonwood/Willow, Aspen, Douglas-fir, Krummholz

In [8]:
# Interactive ML playground for the Forest Covertype dataset (scikit-learn "covtype")
# Single-cell Jupyter code using ipywidgets
# Requirements: scikit-learn, ipywidgets, pandas, numpy
# If widgets do not render, ensure: pip install ipywidgets && jupyter nbextension enable --py widgetsnbextension

import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, balanced_accuracy_score
)
import ipywidgets as widgets
from IPython.display import display, clear_output

#########################################################################################################################
####################################### DATASET #########################################################################
#########################################################################################################################
# -----------------------------
# Load dataset once
# -----------------------------
covtype = fetch_covtype(as_frame=False)
X_all, y_all = covtype.data, covtype.target
X_all = X_all.astype(np.float32)

# Stratified split to preserve class distribution
X, _, y, _ = train_test_split(
    X_all, y_all, train_size=0.1, stratify=y_all, random_state=42
)

# -----------------------------
# Widgets
# -----------------------------
class_groups = {
    "Subset A": [1, 2, 5],
    "Subset B": [4, 5, 7],
    "Subset C": [3, 6, 7],
}

group_dropdown = widgets.Dropdown(
    options=list(class_groups.keys()),
    value="Subset A",
    description="Subset:",
)

viz_points = 150 #widgets.IntSlider(
#    value=600, min=200, max=3000, step=100, description="Max points (pairplot)"
#)

run_button = widgets.Button(description="Pick subset", button_style="primary")

output = widgets.Output()

controls = widgets.VBox([
    widgets.HTML("<h2>Forest Covertype Subset Loading and Visualization</h2>"),
    widgets.HBox([group_dropdown]),
    #widgets.HBox([viz_points, show_pairplot]),
    run_button
])

# -----------------------------
# Helper: balanced subsample per class
# -----------------------------
def balanced_subsample(Xs, ys, max_total=600, seed=42):
    rng = np.random.default_rng(seed)
    classes, counts = np.unique(ys, return_counts=True)
    per_class = max_total // len(classes) if len(classes) > 0 else max_total
    idx_all = []
    for c in classes:
        idx_c = np.where(ys == c)[0]
        take = min(len(idx_c), per_class)
        if take > 0:
            chosen = rng.choice(idx_c, size=take, replace=False)
            idx_all.append(chosen)
    if len(idx_all) == 0:
        return Xs[:0], ys[:0]
    idx_all = np.concatenate(idx_all)
    rng.shuffle(idx_all)
    return Xs[idx_all], ys[idx_all]

# -----------------------------
# Main runner
# -----------------------------
def run_dataset(*args):
    with output:
        clear_output(wait=True)

        selected = class_groups[group_dropdown.value]
        mask = np.isin(y, selected)
        X_sub = X[mask]
        y_sub = y[mask]

        global X_train, X_test, y_train, y_test
        X_train, X_test, y_train, y_test = train_test_split(
            X_sub, y_sub, test_size=0.2, stratify=y_sub, random_state=42
        )

        present_classes = np.unique(y_sub)
        print(f"Requested classes: {selected}")
        print(f"Present classes in dataset: {present_classes.tolist()}")
        if len(X_sub) == 0:
            print("No samples found for the selected class group. Try a different selection.")
            return

        # Summary
        counts = {int(c): int((y_sub == c).sum()) for c in present_classes}
        print(f"Subset size: {len(X_sub)} samples")
        print(f"Class counts: {counts}")

        # Correlation heatmap for first 10 features (across the subset)
        #cols = [f"f{i}" for i in range(10)]
        cols = [
            'Elevation',
            'Aspect',
            'Slope',
            'HDist_Hydro',
            'VDist_Hydro',
            'HDist_Road',
            'Hillsh_9am',
            'Hillsh_Noon',
            'Hillsh_3pm',
            'HDist_FirePts'
        ]
        df10 = pd.DataFrame(X_sub[:, :10], columns=cols)
        corr = df10.corr(method="pearson")

        X_vis, y_vis = balanced_subsample(X_sub, y_sub, max_total=viz_points, seed=42)
        if len(X_vis) == 0:
            print("Not enough samples to create pairplot.")
            return
        df_vis = pd.DataFrame(X_vis[:, :10], columns=cols)
        df_vis["class"] = y_vis.astype(int)

        # Use corner=True to reduce complexity of the grid
        g = sns.pairplot(
            df_vis, vars=cols, hue="class", corner=True,
            plot_kws={"s": 10, "alpha": 0.6},
            diag_kind="kde"
        )
        g.fig.set_size_inches(12, 12)
        g.fig.suptitle("Pairplot (first 10 features, colored by class, subsampled)", y=1.02)
        plt.show()

run_button.on_click(run_dataset)


data_ui = widgets.VBox([controls, output])


#########################################################################################################################
####################################### MODEL ###########################################################################
#########################################################################################################################

# -----------------------------
# Utility: build preprocessor
# -----------------------------
def make_preprocessor(choice):
    if choice == "None":
        return None
    if choice == "Center":
        return StandardScaler(with_mean=True, with_std=False)
    if choice == "Scale":
        return StandardScaler(with_mean=False, with_std=True)
    if choice == "Center & Scale":
        return StandardScaler()
    if choice == "Normalize (L2)":
        return Normalizer(norm="l2")
    raise ValueError(f"Unknown preprocessing option: {choice}")

# -----------------------------
# Utility: balance training set by random undersampling to smallest class count
# -----------------------------
def balance_training(Xt, yt, seed=42):
    rng = np.random.default_rng(seed)
    classes, counts = np.unique(yt, return_counts=True)
    #n = counts.min()
    n = int(counts.mean())
    idx_list = []
    for c in classes:
        idx_c = np.where(yt == c)[0]
        if len(idx_c) < n:
            chosen = rng.choice(idx_c, size=n, replace=True)
        else:
            chosen = rng.choice(idx_c, size=n, replace=False)
        idx_list.append(chosen)
    idx_all = np.concatenate(idx_list)
    rng.shuffle(idx_all)
    return Xt[idx_all], yt[idx_all]

# -----------------------------
# Widgets: Data Preparation
# -----------------------------
preproc_dropdown = widgets.Dropdown(
    options=["None", "Center", "Scale", "Center & Scale", "Normalize (L2)"],
    value="Center & Scale",
    description="Preprocess:",
)

balance_checkbox = widgets.Checkbox(
    value=False, description="Balance training samples"
)

data_prep_box = widgets.VBox([preproc_dropdown, balance_checkbox])

# -----------------------------
# Widgets: Feature Selection (columns 0..9)
# -----------------------------
#feature_checkboxes = [widgets.Checkbox(value=True, description=f"Col {i}") for i in range(10)]
feature_checkboxes = [widgets.Checkbox(value=True, description=f"{i}") for i in covtype.feature_names[:10]]
# Arrange in two rows for readability
feature_selection_box = widgets.VBox([
    widgets.HBox(feature_checkboxes[:3]),
    widgets.HBox(feature_checkboxes[3:6]),
    widgets.HBox(feature_checkboxes[6:9]),
    widgets.HBox(feature_checkboxes[9:]),
])

# -----------------------------
# Widgets: Model Selection
# -----------------------------
model_dropdown = widgets.Dropdown(
    options=["Logistic Regression", "MLP", "Decision Tree", "Random Forest"],
    value="Logistic Regression",
    description="Model:",
)

# Model-dependent complexity controls
mlp_neurons = widgets.IntSlider(value=64, min=8, max=512, step=8, description="# Neurons")
dt_max_depth = widgets.IntSlider(value=20, min=2, max=50, step=1, description="Max depth")
rf_n_estimators = widgets.IntSlider(value=100, min=10, max=300, step=10, description="# Trees")
rf_max_depth = widgets.IntSlider(value=20, min=2, max=50, step=1, description="Max depth")

complexity_box = widgets.VBox([])

def update_complexity_controls(*args):
    mdl = model_dropdown.value
    if mdl == "Logistic Regression":
        complexity_box.children = [widgets.HTML("<i>No complexity slider for Logistic Regression.</i>")]
    elif mdl == "MLP":
        complexity_box.children = [mlp_neurons]
    elif mdl == "Decision Tree":
        complexity_box.children = [dt_max_depth]
    elif mdl == "Random Forest":
        complexity_box.children = [rf_n_estimators, rf_max_depth]

model_dropdown.observe(update_complexity_controls, names="value")
update_complexity_controls()

model_selection_box = widgets.VBox([model_dropdown, complexity_box])

# -----------------------------
# Widgets: Model Training & Hyperparameter Tuning (regularization)
# -----------------------------
# Logistic Regression regularization
lr_C = widgets.FloatLogSlider(
    value=1.0, base=10, min=-2, max=2, step=0.1, description="LR C (1/λ)"
)

# MLP regularization
mlp_alpha = widgets.FloatLogSlider(
    value=1e-4, base=10, min=-6, max=-1, step=0.1, description="MLP α (L2)"
)

# Decision Tree regularization (cost-complexity pruning alpha)
dt_ccp_alpha = widgets.FloatSlider(
    value=0.0, min=0.0, max=0.05, step=0.001, description="DT ccp_alpha"
)

# Random Forest regularization-ish
rf_min_samples_leaf = widgets.IntSlider(
    value=1, min=1, max=20, step=1, description="RF min_samples_leaf"
)

tuning_box = widgets.VBox([])

def update_tuning_controls(*args):
    mdl = model_dropdown.value
    if mdl == "Logistic Regression":
        tuning_box.children = [lr_C]
    elif mdl == "MLP":
        tuning_box.children = [mlp_alpha]
    elif mdl == "Decision Tree":
        tuning_box.children = [dt_ccp_alpha]
    elif mdl == "Random Forest":
        tuning_box.children = [rf_min_samples_leaf]

model_dropdown.observe(update_tuning_controls, names="value")
update_tuning_controls()

# -----------------------------
# Containers with block titles
# -----------------------------
data_prep_section = widgets.VBox([widgets.HTML("<h3>Data Preparation</h3>"), data_prep_box])
feature_selection_section = widgets.VBox([widgets.HTML("<h3>Feature Selection</h3>"), feature_selection_box])
model_selection_section = widgets.VBox([widgets.HTML("<h3>Model Selection</h3>"), model_selection_box])
tuning_section = widgets.VBox([widgets.HTML("<h3>Model Training & Hyperparameter Tuning</h3>"), tuning_box])

# Optionally use Accordion
accordion = widgets.Accordion(children=[
    data_prep_box, feature_selection_box, model_selection_box, tuning_box
])
accordion.set_title(0, "Data Preparation")
accordion.set_title(1, "Feature Selection")
accordion.set_title(2, "Model Selection")
accordion.set_title(3, "Model Training & Hyperparameter Tuning")
#accordion = widgets.VBox([
#    data_prep_box, feature_selection_box, model_selection_box, tuning_box
#])

# -----------------------------
# Output and run controls
# -----------------------------
run_button = widgets.Button(description="Train", button_style="success")
output = widgets.Output()

# -----------------------------
# Build model pipeline based on UI
# -----------------------------
def build_classifier():
    mdl = model_dropdown.value
    if mdl == "Logistic Regression":
        clf = LogisticRegression(
            C=lr_C.value,
            penalty="l2",
            solver="lbfgs",
            max_iter=200,
            #multi_class="auto",
            random_state=42
        )
    elif mdl == "MLP":
        clf = MLPClassifier(
            hidden_layer_sizes=(mlp_neurons.value,),
            activation="relu",
            solver="adam",
            alpha=mlp_alpha.value,
            max_iter=50,
            early_stopping=True,
            random_state=42
        )
    elif mdl == "Decision Tree":
        clf = DecisionTreeClassifier(
            max_depth=dt_max_depth.value,
            ccp_alpha=dt_ccp_alpha.value,
            random_state=42
        )
    elif mdl == "Random Forest":
        clf = RandomForestClassifier(
            n_estimators=rf_n_estimators.value,
            max_depth=rf_max_depth.value,
            min_samples_leaf=rf_min_samples_leaf.value,
            n_jobs=-1,
            random_state=42
        )
    else:
        raise ValueError("Unknown model")
    return clf

# -----------------------------
# Run experiment on button click
# -----------------------------
def run_experiment(*args):
    with output:
        clear_output(wait=True)

        # Feature selection: collect selected columns
        selected_cols = [i for i, cb in enumerate(feature_checkboxes) if cb.value]
        if len(selected_cols) == 0:
            print("Please select at least one feature (column 0..9).")
            return

        # Slice features
        Xtr = X_train[:, selected_cols + list(range(10,54))]
        Xte = X_test[:, selected_cols + list(range(10,54))]

        # Balance training set if requested
        if balance_checkbox.value:
            Xtr_bal, ytr_bal = balance_training(Xtr, y_train, seed=42)
        else:
            Xtr_bal, ytr_bal = Xtr, y_train

        # Build pipeline
        preproc = make_preprocessor(preproc_dropdown.value)
        clf = build_classifier()
        steps = []
        if preproc is not None:
            steps.append(("preprocess", preproc))
        steps.append(("clf", clf))
        pipe = Pipeline(steps)

        # Train
        t0 = time.perf_counter()
        pipe.fit(Xtr_bal, ytr_bal)
        t_train = time.perf_counter() - t0

        # Predict
        t1 = time.perf_counter()
        y_pred = pipe.predict(Xte)
        t_pred = time.perf_counter() - t1

        # Metrics
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average="macro", zero_division=0)
        rec = recall_score(y_test, y_pred, average="macro", zero_division=0)
        f1m = f1_score(y_test, y_pred, average="macro", zero_division=0)
        cm = confusion_matrix(y_test, y_pred)

        # Report
        print("=== Configuration ===")
        print(f"Selected features: {selected_cols} (count={len(selected_cols)})")
        print(f"Preprocessing: {preproc_dropdown.value}")
        print(f"Balanced training: {balance_checkbox.value}")
        print(f"Model: {model_dropdown.value}")
        if model_dropdown.value == "MLP":
            print(f" - Neurons: {mlp_neurons.value}, alpha={mlp_alpha.value:g}")
        elif model_dropdown.value == "Decision Tree":
            print(f" - Max depth: {dt_max_depth.value}, ccp_alpha={dt_ccp_alpha.value:g}")
        elif model_dropdown.value == "Random Forest":
            print(f" - Trees: {rf_n_estimators.value}, Max depth: {rf_max_depth.value}, min_samples_leaf={rf_min_samples_leaf.value}")
        elif model_dropdown.value == "Logistic Regression":
            print(f" - C (1/λ): {lr_C.value:g}")

        print("\n=== Performance (test set) ===")
        print(f"Accuracy: {acc:.4f}")
        print(f"Precision (macro): {prec:.4f}")
        print(f"Recall (macro): {rec:.4f}")
        print(f"F1-score (macro): {f1m:.4f}")
        print(f"Balanced accuracy: {balanced_accuracy_score(y_test, y_pred):.4f}")
        print(f"\nTraining time: {t_train:.3f} s")
        print(f"Inference time (predict): {t_pred:.3f} s")

        #print("\nConfusion matrix (rows: true, cols: predicted):")
        #cm_df = pd.DataFrame(cm)
        #display(cm_df)
        
        print("\nConfusion matrix:")
        labels = np.unique(y_test)
        fig, ax = plt.subplots(figsize=(5, 5), dpi=120)
        disp = ConfusionMatrixDisplay.from_predictions(
            y_test, y_pred, cmap="Blues", normalize='true', values_format='.3f', colorbar=True, ax=ax
        )
        ax.set_title("Confusion Matrix")
        plt.tight_layout()
        plt.show()

run_button.on_click(run_experiment)

# -----------------------------
# Display UI
# -----------------------------
ui = widgets.VBox([
    widgets.HTML("<h2>Forest Covertype ML Playground</h2>"),
    accordion,
    widgets.HBox([run_button]),
    output
])

display(widgets.VBox([data_ui, ui]))